In [1]:
import os
import kagglehub
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import classification_report
from tqdm import tqdm

In [2]:
path = kagglehub.dataset_download("ankit1743/skyview-an-aerial-landscape-dataset")
print("Path to dataset files:", path)

DATA_DIR = os.path.join(path, "Aerial_Landscapes")# Confirm if folder name is "SkyView"
BATCH_SIZE = 16
NUM_CLASSES = 15
EPOCHS = 10
DEVICE = torch.device("cuda")

Path to dataset files: /kaggle/input/skyview-an-aerial-landscape-dataset


In [3]:
train_transforms = transforms.Compose([
    transforms.Resize((128, 128)),  
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
])
test_transforms = transforms.Compose([
    transforms.Resize((128, 128)), 
    transforms.ToTensor(),
])

In [4]:
def get_model():
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
    return model.to(DEVICE)

In [5]:
def train_model(model, train_loader, test_loader):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(model(imgs), labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Loss: {total_loss / len(train_loader):.4f}")
        evaluate_model(model, test_loader)

In [6]:
def evaluate_model(model, loader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(DEVICE)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    print(classification_report(y_true, y_pred, zero_division=0))

In [7]:
def main():
    dataset = datasets.ImageFolder(DATA_DIR, transform=train_transforms)
    train_len = int(0.8 * len(dataset))
    test_len = len(dataset) - train_len
    train_data, test_data = random_split(dataset, [train_len, test_len])

    train_data.dataset.transform = train_transforms
    test_data.dataset.transform = test_transforms

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

    model = get_model()
    train_model(model, train_loader, test_loader)

if __name__ == '__main__':
    main()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 176MB/s]
Epoch 1/10: 100%|██████████| 600/600 [01:15<00:00,  8.00it/s]


Loss: 0.5097
              precision    recall  f1-score   support

           0       0.97      0.94      0.96       171
           1       0.90      0.87      0.88       162
           2       0.92      0.97      0.95       157
           3       0.95      0.88      0.91       158
           4       0.96      0.93      0.95       153
           5       0.95      0.96      0.96       169
           6       0.96      0.98      0.97       156
           7       0.94      0.98      0.96       150
           8       0.93      0.97      0.95       146
           9       0.93      0.90      0.91       148
          10       0.99      0.99      0.99       164
          11       0.94      0.99      0.97       169
          12       0.90      0.95      0.92       169
          13       0.94      0.99      0.97       162
          14       0.97      0.86      0.91       166

    accuracy                           0.94      2400
   macro avg       0.94      0.94      0.94      2400
weighted avg 

Epoch 2/10: 100%|██████████| 600/600 [00:28<00:00, 20.93it/s]


Loss: 0.1833
              precision    recall  f1-score   support

           0       0.99      0.95      0.97       171
           1       0.87      0.92      0.89       162
           2       0.96      0.96      0.96       157
           3       0.86      0.97      0.91       158
           4       0.93      0.97      0.95       153
           5       0.92      0.97      0.95       169
           6       0.97      0.97      0.97       156
           7       0.94      0.94      0.94       150
           8       0.96      0.99      0.98       146
           9       0.95      0.91      0.93       148
          10       0.96      0.97      0.97       164
          11       0.96      0.97      0.96       169
          12       0.95      0.79      0.86       169
          13       0.98      0.99      0.98       162
          14       0.98      0.90      0.94       166

    accuracy                           0.94      2400
   macro avg       0.95      0.94      0.94      2400
weighted avg 

Epoch 3/10: 100%|██████████| 600/600 [00:28<00:00, 20.79it/s]


...

Loss: 0.0443
              precision    recall  f1-score   support

           0       0.99      0.91      0.95       171
           1       0.90      0.95      0.92       162
           2       0.97      0.97      0.97       157
           3       0.92      0.89      0.91       158
           4       0.95      0.97      0.96       153
           5       0.93      0.98      0.95       169
           6       0.95      0.96      0.96       156
           7       0.91      0.97      0.94       150
           8       0.95      0.99      0.97       146
           9       0.89      0.95      0.92       148
          10       0.98      0.96      0.97       164
          11       0.98      0.96      0.97       169
          12       0.96      0.90      0.93       169
          13       0.95      0.92      0.93       162
          14       0.96      0.91      0.93       166

    accuracy                           0.95      2400
   macro avg       0.95      0.95      0.95      2400
weighted avg 

Epoch 9/10: 100%|██████████| 600/600 [00:28<00:00, 21.05it/s]


Loss: 0.0473
              precision    recall  f1-score   support

           0       0.98      0.94      0.96       171
           1       0.88      0.89      0.89       162
           2       0.94      0.99      0.96       157
           3       0.91      0.94      0.93       158
           4       0.98      0.95      0.97       153
           5       0.96      0.98      0.97       169
           6       0.93      0.98      0.96       156
           7       0.88      0.97      0.92       150
           8       0.95      0.99      0.97       146
           9       0.90      0.97      0.93       148
          10       0.99      0.97      0.98       164
          11       0.99      0.92      0.95       169
          12       0.96      0.92      0.94       169
          13       0.99      0.95      0.97       162
          14       0.97      0.86      0.91       166

    accuracy                           0.95      2400
   macro avg       0.95      0.95      0.95      2400
weighted avg 

Epoch 10/10: 100%|██████████| 600/600 [00:29<00:00, 20.46it/s]


Loss: 0.0369
              precision    recall  f1-score   support

           0       0.98      0.95      0.96       171
           1       0.92      0.93      0.92       162
           2       0.96      0.96      0.96       157
           3       0.96      0.94      0.95       158
           4       0.95      0.97      0.96       153
           5       0.88      0.96      0.92       169
           6       0.97      0.87      0.92       156
           7       0.98      0.88      0.93       150
           8       0.95      0.99      0.97       146
           9       0.92      0.95      0.93       148
          10       1.00      0.95      0.97       164
          11       0.96      0.97      0.96       169
          12       0.93      0.96      0.94       169
          13       0.94      0.97      0.95       162
          14       0.92      0.93      0.92       166

    accuracy                           0.95      2400
   macro avg       0.95      0.95      0.95      2400
weighted avg 